In [17]:
# 导入相关的包
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import os
import torch.nn.functional as F
# 指定训练设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
# 对图片的预处理
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256, 256)),
    torchvision.transforms.RandomCrop((224, 224)),
    torchvision.transforms.ToTensor(),
    normalize
])

# 导入数据集
train_data = torchvision.datasets.CIFAR10(root='../数据集/cifar10', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='../数据集/cifar10', train=False, transform=transform, download=True)


Files already downloaded and verified
Files already downloaded and verified


In [19]:
# 保存数据集的长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

训练数据集的长度为：50000
测试数据集的长度为：10000


In [20]:
# 创建数据加载器
train_dataloader = DataLoader(train_data, batch_size=256)
test_dataloader = DataLoader(test_data, batch_size=256)

In [21]:
# 加载预训练模型
resnet18 = torchvision.models.resnet18(pretrained=True)
# 将模型移动到GPU上
resnet18 = resnet18.to(device)
resnet18.fc

Linear(in_features=512, out_features=1000, bias=True)

In [22]:
# 修改最后一层以适应CIFAR-10的类别数
num_ftrs = resnet18.fc.in_features
resnet18.fc = torch.nn.Linear(num_ftrs, 10)

In [23]:
# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

# 优化器
learning_rate = 0.0001
optimizer = torch.optim.SGD(resnet18.parameters(), lr=learning_rate,momentum=0.9)

In [24]:
# 冻结所有层，只训练最后一层
for param in resnet18.parameters():
    param.requires_grad = False
resnet18.fc.requires_grad = True

In [25]:
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0

# 设置训练网络的循环次数
epoch = 5

# 开始训练
for i in range(epoch):
    # 训练过程
    print("------第{}轮训练开始------".format(i + 1))
    resnet18.train()
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = resnet18(imgs)
        loss = loss_fn(output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数：{}，loss：{}".format(total_train_step, loss.item()))

    # 测试过程
    resnet18.eval()
    with torch.no_grad():
        total_test_loss = 0
        total_accuracy = 0
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            output = resnet18(imgs)
            loss = loss_fn(output, targets)
            total_test_loss += loss.item()
            accuracy = (output.argmax(1) == targets).sum()
            total_accuracy += accuracy.item()
            total_test_step += 1

    # 输出一个轮次的信息
    print(f"整体测试集上的loss:{total_test_loss}")
    print(f"整体测试集上的正确率:{total_accuracy / test_data_size}")
    total_test_step += 1

------第1轮训练开始------


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)